In [222]:
import numpy as np
import copy
from tqdm import tqdm
import pandas as pd
import re
import gensim
import keras
from keras import optimizers
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.layers import *
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from score import report_score
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [223]:
datadir="fnc-1"
w2v_path = './w2v/GoogleNews-vectors-negative300.bin'
save_path = "./saved/"
batch_size = 128
max_sent_length = 350
random_state = 37
epoch = 10

In [224]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


### Pre-process Dataset

In [225]:
raw_train_bodies = pd.read_csv(datadir + '/train_bodies.csv')   
raw_train_stances = pd.read_csv(datadir + '/train_stances.csv')
raw_test_bodies = pd.read_csv(datadir + '/competition_test_bodies.csv') 
raw_test_stances = pd.read_csv(datadir + '/competition_test_stances.csv') 

In [226]:
stance_to_int = {"agree":0, "discuss": 1, "disagree": 2, "unrelated": 3}
int_to_stance = {0:"agree", 1:"discuss", 2:"disagree", 3: "unrelated"}

In [227]:
actual_test_stances = raw_test_stances['Stance']
raw_train_stances['Stance'] = raw_train_stances['Stance'].apply(lambda x: stance_to_int[x])
raw_test_stances['Stance'] = raw_test_stances['Stance'].apply(lambda x: stance_to_int[x])

In [228]:
train_df = raw_train_stances.join(raw_train_bodies.set_index('Body ID'), on='Body ID')
test_df = raw_test_stances.join(raw_test_bodies.set_index('Body ID'), on='Body ID')

In [229]:
def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

# Pre-processing words
clean_train_headline = [text_to_word_sequence(clean(head)) for head in train_df['Headline']]
clean_train_bodies = [text_to_word_sequence(clean(body)) for body in train_df['articleBody']]
clean_test_headline = [text_to_word_sequence(clean(head)) for head in test_df['Headline']]
clean_test_bodies = [text_to_word_sequence(clean(body)) for body in test_df['articleBody']]

In [230]:
wordlist = []
for i in range(len(clean_train_headline)):
    wordlist.append(clean_train_headline[i])
for i in range(len(clean_train_bodies)):
    wordlist.append(clean_train_bodies[i])
for i in range(len(clean_test_headline)):
    wordlist.append(clean_test_headline[i])
for i in range(len(clean_test_bodies)):
    wordlist.append(clean_test_bodies[i])

In [231]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(wordlist)
len(tokenizer.word_index)

29451

In [232]:
train_lines = [] 
for i in range(len(clean_train_headline)):
    headline =  clean_train_headline[i]
    body = clean_train_bodies[i]
    newline = headline+body
    train_lines.append(newline)

test_lines = [] 
for i in range(len(clean_test_headline)):
    headline =  clean_test_headline[i]
    body = clean_train_bodies[i]
    newline = headline+body
    test_lines.append(newline)

In [233]:
X_train = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in train_lines])
raw_X_train = pad_sequences(X_train, maxlen=max_sent_length, padding='post', truncating='post')
raw_y_train = train_df['Stance']

In [234]:
X_test = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in test_lines])
X_test = pad_sequences(X_test, maxlen=max_sent_length, padding='post', truncating='post')
y_test = test_df['Stance']

In [235]:
X_train, X_vali, y_train, y_vali = train_test_split(raw_X_train, raw_y_train, random_state = random_state, test_size=0.2)

In [236]:
# Convert y to onehot
y_train_onehot = np_utils.to_categorical(y_train)
y_vali_onehot = np_utils.to_categorical(y_vali)
y_test_onehot = np_utils.to_categorical(y_test)

### Basic Model

In [237]:
embedding_dim = 300
embeddings = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, embedding_dim))

for word, i in tokenizer.word_index.items():
    try:
        embeddings_vector = embeddings[word]
        embeddings_matrix[i] = embeddings_vector
    except KeyError:
        pass
        
del embeddings


In [238]:
def cnn_model(n_classes):
    kernel_sizes = [3, 4, 5]
    num_filters=[80, 80, 80]  
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                            output_dim=embedding_dim,
                            weights = [embeddings_matrix],
                            trainable=False, name='embedding_layer',
                            mask_zero=True))
    model.add(keras.layers.Conv1D(num_filters[0], kernel_sizes[0], padding='valid', activation='relu', kernel_regularizer=regularizers.L2(0.001)))
    model.add(Activation(activation='relu', name='activation_1'))
    model.add(keras.layers.MaxPooling1D(3))
    model.add(keras.layers.Conv1D(num_filters[1], kernel_sizes[1], padding='valid', activation='relu', kernel_regularizer=regularizers.L2(0.001)))
    model.add(Activation(activation='relu', name='activation_2'))
    model.add(keras.layers.MaxPooling1D(3))
    model.add(keras.layers.Conv1D(num_filters[2], kernel_sizes[2], padding='valid', activation='relu', kernel_regularizer=regularizers.L2(0.001)))
    model.add(Activation(activation='relu', name='activation_3'))
    model.add(GlobalMaxPooling1D())
    model.add(keras.layers.Dense(n_classes, activation='softmax', name='output_layer'))
    
    return model

### Basic model trained over to Stance Dataset

In [239]:
basic_model = cnn_model(n_classes=4)
basic_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [240]:
print(basic_model.summary())

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 300)         8835600   
_________________________________________________________________
conv1d_57 (Conv1D)           (None, None, 80)          72080     
_________________________________________________________________
activation_1 (Activation)    (None, None, 80)          0         
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, None, 80)          0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, None, 80)          25680     
_________________________________________________________________
activation_2 (Activation)    (None, None, 80)          0         
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, None, 80)        

In [241]:
history = basic_model.fit(X_train, y_train_onehot,
          batch_size=batch_size,
          epochs=epoch,
          validation_data=(X_vali, y_vali_onehot))
basic_model.save(save_path+"basic_mode")

Epoch 1/10
313/313 [==============================] - 4s 13ms/step - loss: 0.8955 - accuracy: 0.7377 - val_loss: 0.7742 - val_accuracy: 0.7560
Epoch 2/10
313/313 [==============================] - 4s 12ms/step - loss: 0.6874 - accuracy: 0.7770 - val_loss: 0.6350 - val_accuracy: 0.7964
Epoch 3/10
313/313 [==============================] - 4s 12ms/step - loss: 0.5830 - accuracy: 0.8228 - val_loss: 0.5711 - val_accuracy: 0.8334
Epoch 4/10
313/313 [==============================] - 4s 12ms/step - loss: 0.5289 - accuracy: 0.8494 - val_loss: 0.5623 - val_accuracy: 0.8402
Epoch 5/10
313/313 [==============================] - 4s 12ms/step - loss: 0.4900 - accuracy: 0.8690 - val_loss: 0.5328 - val_accuracy: 0.8492
Epoch 6/10
313/313 [==============================] - 4s 12ms/step - loss: 0.4640 - accuracy: 0.8802 - val_loss: 0.5173 - val_accuracy: 0.8585
Epoch 7/10
313/313 [==============================] - 4s 12ms/step - loss: 0.4452 - accuracy: 0.8901 - val_loss: 0.5084 - val_accuracy: 0.8654

In [242]:
# Accuracies on the validation set
preds_vali = np.argmax(basic_model.predict(X_vali), axis = -1)
true_valid = np.argmax(y_vali_onehot, axis = -1)

total_relatedness = len(true_valid)
total_opinion = 0
correct_relatedness = 0
correct_opinion = 0

for i in range(len(true_valid)):
    true = true_valid[i]
    pred = preds_vali[i]
    if true==3:
        if pred==3:
            correct_relatedness+=1
    else:
        total_opinion+=1
        if pred!=3:
            correct_relatedness+=1
        if pred==true:
            correct_opinion+=1

print(f"The Relatedness Accuracy is {correct_relatedness/total_relatedness}")
print(f"The Opinion Accuracy is {correct_opinion/total_opinion}")

The Relatedness Accuracy is 0.895847923961981
The Opinion Accuracy is 0.6226277372262774


In [243]:
preds = basic_model.predict(X_test)
outputs = [int_to_stance[np.argmax(p, axis = -1)] for p in preds]
report_score(actual_test_stances,outputs)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    174    |     9     |    101    |   1619    |
-------------------------------------------------------------
| disagree  |    54     |     5     |    33     |    605    |
-------------------------------------------------------------
|  discuss  |    255    |    19     |    314    |   3876    |
-------------------------------------------------------------
| unrelated |   1321    |    87     |   1184    |   15757   |
-------------------------------------------------------------
Score: 4550.0 out of 11651.25	(39.05160390516039%)


39.05160390516039

### Relatedness Classifier

In [244]:
# agree, disagree, discuss are 1, unrelated is 0
int_to_relatedness={0:1,1:1,2:1,3:0}
str_to_relatedness = {'unrelated':0 , 'related':1}

In [245]:
relatedness_y_train = y_train.apply(lambda x: int_to_relatedness[x])
relatedness_y_train_onehot = np_utils.to_categorical(relatedness_y_train)

relatedness_y_vali = y_vali.apply(lambda x: int_to_relatedness[x])
relatedness_y_vali_onehot = np_utils.to_categorical(relatedness_y_vali)

relatedness_y_test = y_test.copy()
relatedness_y_test = relatedness_y_test.apply(lambda x: int_to_relatedness[x])

In [246]:
relatedness_model = cnn_model(n_classes=2)
relatedness_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [247]:
history = relatedness_model.fit(X_train, relatedness_y_train_onehot ,batch_size=batch_size,epochs=epoch, 
                                validation_data=(X_vali, relatedness_y_vali_onehot))
preds = relatedness_model.predict(X_test)
outputs = [np.argmax(p, axis = -1) for p in preds]

Epoch 1/10
313/313 [==============================] - 4s 13ms/step - loss: 0.6703 - accuracy: 0.7421 - val_loss: 0.5516 - val_accuracy: 0.7972
Epoch 2/10
313/313 [==============================] - 4s 12ms/step - loss: 0.4994 - accuracy: 0.8156 - val_loss: 0.4756 - val_accuracy: 0.8333
Epoch 3/10
313/313 [==============================] - 4s 12ms/step - loss: 0.4276 - accuracy: 0.8548 - val_loss: 0.4298 - val_accuracy: 0.8489
Epoch 4/10
313/313 [==============================] - 4s 12ms/step - loss: 0.3852 - accuracy: 0.8759 - val_loss: 0.4380 - val_accuracy: 0.8511
Epoch 5/10
313/313 [==============================] - 4s 12ms/step - loss: 0.3590 - accuracy: 0.8929 - val_loss: 0.4006 - val_accuracy: 0.8679
Epoch 6/10
313/313 [==============================] - 4s 12ms/step - loss: 0.3363 - accuracy: 0.9052 - val_loss: 0.3758 - val_accuracy: 0.8892
Epoch 7/10
313/313 [==============================] - 4s 13ms/step - loss: 0.3208 - accuracy: 0.9139 - val_loss: 0.3701 - val_accuracy: 0.8915

### Opinion Classfier

#### Dataset that all unrelated column is removed

In [248]:
train_drop_index = train_df[train_df['Stance']==3].index
opinion_train_df = train_df.drop(train_df[train_df['Stance']==3].index)

test_drop_index = test_df[test_df['Stance']==3].index
opinion_test_df = test_df.drop(test_df[test_df['Stance']==3].index)

In [249]:
opinion_train_headline = [text_to_word_sequence(clean(head)) for head in opinion_train_df['Headline']]
opinion_train_bodies = [text_to_word_sequence(clean(body)) for body in opinion_train_df['articleBody']]

opinion_test_headline = [text_to_word_sequence(clean(head)) for head in opinion_test_df['Headline']]
opinion_test_bodies = [text_to_word_sequence(clean(body)) for body in opinion_test_df['articleBody']]

In [250]:
opinion_train_lines = [] 
for i in range(len(opinion_train_headline)):
    headline =  opinion_train_bodies[i]
    body = clean_train_bodies[i]
    newline = headline+body
    opinion_train_lines.append(newline)

opinion_test_lines = [] 
for i in range(len(opinion_test_headline)):
    headline =  opinion_test_bodies[i]
    body = clean_train_bodies[i]
    newline = headline+body
    opinion_test_lines.append(newline)

In [251]:
len(opinion_train_lines)

13427

In [252]:
opinion_X_train = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in opinion_train_lines])
opinion_raw_X_train = pad_sequences(opinion_X_train, maxlen=max_sent_length, padding='post', truncating='post')
opinion_raw_y_train = opinion_train_df['Stance']

In [253]:
opinion_X_test = tokenizer.texts_to_sequences([' '.join(seq[:max_sent_length]) for seq in opinion_test_lines])
opinion_raw_X_test = pad_sequences(opinion_X_test, maxlen=max_sent_length, padding='post', truncating='post')
opinion_y_test = opinion_test_df['Stance']

In [254]:
# Convert y to onehot
opinion_y_train_onehot = np_utils.to_categorical(opinion_raw_y_train)
opinion_y_test_onehot = np_utils.to_categorical(opinion_y_test)

In [255]:
opinion_X_train, opinion_X_vali, opinion_y_train, opinion_y_vali = train_test_split(opinion_raw_X_train, opinion_y_train_onehot, random_state=random_state, test_size=0.2)

In [256]:
opinion_model = cnn_model(n_classes=3)
opinion_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [257]:
history = opinion_model.fit(opinion_X_train, opinion_y_train,
          batch_size=batch_size,
          epochs=epoch,
          validation_data=(opinion_X_vali, opinion_y_vali))
#basic_model.save(save_path+"opinion_model")

Epoch 1/10
84/84 [==============================] - 2s 14ms/step - loss: 1.0053 - accuracy: 0.6661 - val_loss: 0.9053 - val_accuracy: 0.7200
Epoch 2/10
84/84 [==============================] - 1s 12ms/step - loss: 0.8440 - accuracy: 0.7241 - val_loss: 0.8178 - val_accuracy: 0.7398
Epoch 3/10
84/84 [==============================] - 1s 13ms/step - loss: 0.7688 - accuracy: 0.7539 - val_loss: 0.7405 - val_accuracy: 0.7599
Epoch 4/10
84/84 [==============================] - 1s 12ms/step - loss: 0.7152 - accuracy: 0.7686 - val_loss: 0.7310 - val_accuracy: 0.7789
Epoch 5/10
84/84 [==============================] - 1s 12ms/step - loss: 0.6806 - accuracy: 0.7834 - val_loss: 0.6759 - val_accuracy: 0.7755
Epoch 6/10
84/84 [==============================] - 1s 13ms/step - loss: 0.6334 - accuracy: 0.8001 - val_loss: 0.6718 - val_accuracy: 0.7867
Epoch 7/10
84/84 [==============================] - 1s 13ms/step - loss: 0.6021 - accuracy: 0.8087 - val_loss: 0.7704 - val_accuracy: 0.7368
Epoch 8/10
84

In [258]:
preds = opinion_model.predict(opinion_raw_X_test)
outputs = [np.argmax(p, axis = -1) for p in preds]
accuracy_score(opinion_y_test, outputs)

0.5863533408833522

### Combine Relatedness model and Opinion Model

In [259]:
class cascaded_model:
    def __init__(self,relatedness_model, opinion_model):
        self.relatedness_model = relatedness_model
        self.opinion_model = opinion_model

    def predict(self, X_test):
        prediction = relatedness_model.predict(X_test)
        prediction = [np.argmax(p, axis = -1) for p in prediction]
        for i in tqdm(range(len(prediction))):
            relatedness = prediction[i]
            if relatedness == 1: #related
                opinion = opinion_model.predict(np.array([X_test[i]]))
                opinion = np.argmax(opinion, axis = -1)
                
                prediction[i] = int(opinion)
            else:
                prediction[i] = 3

        return prediction

In [260]:
cascaded = cascaded_model(relatedness_model, opinion_model)
preds_vali = cascaded.predict(X_vali)
true_valid = np.argmax(y_vali_onehot, axis = -1)
print(f"The stance accuracy of cascaded model is {accuracy_score(true_valid,preds_vali)}")

100%|██████████| 9995/9995 [01:19<00:00, 126.33it/s]

The stance accuracy of cascaded model is 0.8604302151075538


: 